<a href="https://colab.research.google.com/github/thisisrach/ANN/blob/main/artificial_neural_network_rj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

In [2]:
tf.__version__


'2.14.0'

In [4]:
# 1.  Data Preprocessing

dataset=pd.read_csv('Churn_Modelling.csv')
x=dataset.iloc[:,3:-1].values
y=dataset.iloc[:,-1].values

In [5]:
print(x)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [6]:
print(y)

[1 0 1 ... 1 1 0]


In [7]:
# Take care of the missing data
# (For that u shld write the code)

# Encoding the Categorical Data (Dependent Variable)

from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
x[:,2]=le.fit_transform(x[:,2])

In [8]:
print(x)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


In [9]:
# Encoding the independent variable
# One Hot Encode

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct=ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[1])],remainder='passthrough')
x=np.array(ct.fit_transform(x))



In [10]:
print(x)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [11]:
# Split the dataset into training set and test set

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [12]:
# Feature Scaling

from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [13]:
# 2. Building the ANN

# Initializing the ANN

ann=tf.keras.models.Sequential()

In [14]:
# Add the input layer and the first hidden layer

ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

In [15]:
# Add the second hidden layer

ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

In [16]:
# Add the output layer

ann.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

# For non binary classification use 'softmax'

In [17]:
# 3. Training the ANN

In [18]:
# Compiling the ANN

ann.compile(optimizer = 'adam',loss = 'binary_crossentropy',metrics = ['accuracy'])

# 'adam' optimizer probably is used for stochastic gradient descent
# For non binary classification use 'categorical_crossentropy'

In [19]:
# Training the ANN

ann.fit(X_train,y_train,batch_size=32,epochs=100)

Epoch 1/100
250/250 [==============================] - 2s 2ms/step - loss: 0.6097 - accuracy: 0.6946
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4749 - accuracy: 0.7990
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4463 - accuracy: 0.8077
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4316 - accuracy: 0.8114
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4225 - accuracy: 0.8204
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4168 - accuracy: 0.8225
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4127 - accuracy: 0.8255
Epoch 8/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4093 - accuracy: 0.8257
Epoch 9/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4061 - accuracy: 0.8282
Epoch 10/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4032 - accura

In [20]:
# 4. Making predictions and evaluating the model

In [23]:
print(ann.predict(sc.transform([[1,0,0,600,1,40,3,60000,2,1,1,50000]])) > 0.5)

1/1 [==============================] - 0s 48ms/step
[[False]]
